#step1 : 데이터 업로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/dataset.zip

# step2 : YOLOv5 클론 만들기

In [ ]:
# yolov5 모델 설치 -> github주소를 클론으로 만듬.

%cd /content/drive/MyDrive/project
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# yolov5 환경 세팅
%cd /content/drive/MyDrive/project/yolov5
!pip install -qr requirements.txt

#step3 : 이미지 및 라벨링 개수 확인

In [ ]:
# 데이터 확인하기
from glob import glob
import torch
from IPython.display import Image, clear_output

img_list = glob('/content/dataset/*.jpg')
print(len(img_list))

txt_list = glob('/content/dataset/*.txt')
print(len(txt_list))

In [ ]:
!pwd

In [ ]:
import os

os.mkdir('/content/drive/MyDrive/project/yolov5/project_custom')
os.mkdir('/content/drive/MyDrive/project/yolov5/project_custom/images')
os.mkdir('/content/drive/MyDrive/project/yolov5/project_custom/labels')

In [ ]:
# 이미지 파일 이동
import shutil

for img in img_list:
  shutil.copy(img, './project_custom/images')

In [ ]:
# 텍스트 파일 이동
import shutil

for txt in txt_list:
  shutil.copy(txt, './project_custom/labels')

#step4 : train을 위한 형태로 변환

In [ ]:
train_img_list = glob('/content/drive/MyDrive/project/yolov5/project_custom/images/*.jpg')
with open('/content/drive/MyDrive/project/yolov5/project_custom/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

In [ ]:
# IPython에서 입력된 내용을 텍스트처럼 계속 사용하기 위해 정의
from IPython.core.magic import register_line_cell_magic

# register_line_cell_magic을 이용해 IPython에서 바로 사용할 수 있게 정의
# writetimplate를 쉽게 사용할 수 있음
@register_line_cell_magic
def writetemplate(line, cell):
  with open(line, 'w') as f:
    f.write(cell.format(**globals()))

#step5 : 데이터 yaml 파일 생성

In [ ]:
# yaml 파일 만들기
%%writetemplate /content/drive/MyDrive/project/yolov5/project_custom/data.yaml


train: /content/drive/MyDrive/project/yolov5/project_custom/images
val: /content/drive/MyDrive/project/yolov5/project_custom/images

nc: 2
names: ['red',
'white']

In [ ]:
%cat /content/drive/MyDrive/project/yolov5/project_custom/data.yaml

In [ ]:
import yaml

with open('/content/drive/MyDrive/project/yolov5/project_custom/data.yaml','r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

print(num_classes)

#step6 : 가중치 다운받기

In [ ]:
!cat /content/drive/MyDrive/project/yolov5/models/yolov5s.yaml

In [ ]:
%%writetemplate /content/drive/MyDrive/project/yolov5/models/custom_yolov5s.yaml
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
!cat /content/drive/MyDrive/project/yolov5/models/custom_yolov5s.yaml

#step 7 : 학습하기

In [ ]:
%cd /content/drive/MyDrive/project/yolov5
!ls

In [ ]:
# 데이터 학습하기
!python /content/drive/MyDrive/project/yolov5/train.py --img 512 --batch 16 --epoch 100 --data /content/drive/MyDrive/project/yolov5/project_custom/data.yaml --cfg /content/drive/MyDrive/project/yolov5/models/custom_yolov5s.yaml --weight yolov5s.pt --name ./_custom/project_results --cache --device 0

#function CaptchaButton(){ console.log("Captcha resolved");
document.getElementsByClassName("recaptcha-checkbox goog-inline-block recaptcha-checkbox-unchecked rc-anchor-checkbox").item("recaptcha-anchor").click() } setInterval(CaptchaButton,600000);

In [ ]:
# 진행 상황 확인
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/project/yolov5/runs/train/_custom/project_results4